In [1]:
!pip install wheel==0.37.1 setuptools==59.6.0
!pip install gym[atari,accept-rom-license]==0.21.0 tensorflow-probability==0.19.0 tqdm==4.64.1

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
from typing import List, Callable, Union, Iterable, Any, Tuple
from dataclasses import dataclass, field

import gym
import numpy as np
from PIL import Image
from tqdm import tqdm
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from keras import Model, Input
from keras.layers import \
    Dense, Conv2D, Conv2DTranspose, Dropout, Flatten, \
    Reshape, Lambda, Concatenate, GRU, Layer, Softmax
from keras.optimizers import Adam
from keras.losses import MSE, kullback_leibler_divergence as KLDiv

RenderSubscriber = Callable[[np.ndarray, np.ndarray], None]
TrajectorySubscriber = Callable[[np.ndarray, np.ndarray, np.ndarray], None]

In [3]:
@dataclass
class DreamerSettings:
    action_dims: List[int]
    obs_dims: List[int]
    obs_enc_dims: List[int]
    repr_dims: List[int]
    hidden_dims: List[int]
    dropout_rate: float = 0.2
    committment_cost: float = 0.25

    @property
    def repr_dims_flat(self) -> int:
        return self.repr_dims[0] * self.repr_dims[1]

    @property
    def repr_out_dims_flat(self) -> int:
        return self.repr_dims[0] * self.repr_dims[1] + self.hidden_dims[0]

In [4]:
class STOneHotCategorical(Layer):
    def __init__(self, dims: Union[int, List[int]], name: str="st_cat_onehot"):
        super(STOneHotCategorical, self).__init__(name=name)
        self.softmax = Softmax()
        # self.softmax = Lambda(lambda x: tf.nn.log_softmax(x + 1e-8))
        self.sample = tfp.layers.OneHotCategorical(dims)
        self.stop_grad = Lambda(lambda x: tf.stop_gradient(x))

    def call(self, logits):
        samples = self.sample(logits)
        probs = self.softmax(logits)
        return samples + probs - self.stop_grad(probs)


class VQCodebook(Layer):
    """Representing a codebook of a vector quantization for a given amount
    of classifications with a given amount of classes each. The embedding
    vectors are initialized to match the inputs to be quantized. When calling
    this layer, it expects to receive one-hot encoded categoricals of shape
    (batch_size, num_classifications, num_classes)."""

    def __init__(
            self, num_classifications: int, num_classes: int,
            name: str="vq_codebook"):
        super(VQCodebook, self).__init__(name=name)
        self.num_classifications = num_classifications
        self.num_classes = num_classes
        self.num_embeddings = num_classifications * num_classes
        self.reshape_out = None

    def init_codebook(self, input_shape: Iterable[int]):
        orig_input_shape = input_shape[1:]
        self.reshape_out = Reshape(orig_input_shape)
        input_dims_flat = tf.reduce_prod(orig_input_shape)
        self.embedding_dims = input_dims_flat // self.num_classifications

        if input_dims_flat % self.num_classifications != 0:
            raise ValueError((
                f"The input dimensions {input_dims_flat} must be divisible "
                f"by the number of classifications {self.num_classifications} "
                f"to support swapping each of the {self.num_classifications} slices "
                "from the input vector with a quantized vector from the codebook."))

        embed_shape = (self.embedding_dims, self.num_embeddings)
        self.embeddings = self.add_weight(
            "embeddings", shape=embed_shape, trainable=True, initializer="random_normal")

    def call(self, categoricals_onehot: tf.Tensor):
        categoricals_sparse = tf.argmax(categoricals_onehot, axis=2)
        id_offsets = tf.range(0, self.num_classifications, dtype=tf.int64) * self.num_classes
        categoricals_embed_sparse = categoricals_sparse + id_offsets
        categoricals_embed = tf.one_hot(categoricals_embed_sparse, depth=self.num_embeddings)
        quantized = tf.matmul(categoricals_embed, self.embeddings, transpose_b=True)
        return self.reshape_out(quantized)

    def most_similar_embeddings(self, inputs: tf.Tensor):
        input_shape = (-1, self.num_classifications, self.embedding_dims)
        embed_shape = (-1, self.num_classifications, self.num_classes)
        inputs_per_classification = tf.reshape(inputs, input_shape)
        embeddings_per_classification = tf.reshape(self.embeddings, embed_shape)
        codebook_ids = []

        for i in range(self.num_classifications):
            embeddings = embeddings_per_classification[:, i, :]
            inputs_classif = inputs_per_classification[:, i, :]

            inputs_sqsum = tf.reduce_sum(inputs_classif ** 2, axis=1, keepdims=True)
            embed_sqsum = tf.reduce_sum(embeddings ** 2, axis=0)
            similarity = tf.matmul(inputs_classif, embeddings)
            distances = inputs_sqsum + embed_sqsum - 2 * similarity

            class_ids = tf.argmin(distances, axis=1, output_type=tf.int64)
            codebook_ids.append(tf.expand_dims(class_ids, axis=0))

        codebook_ids = tf.concat(codebook_ids, axis=0)
        codebook_ids = tf.transpose(codebook_ids, perm=[1, 0])
        return codebook_ids


class VQCategorical(Layer):
    """Representing a transformation of an input vector to be quantized into
    a one-hot encoded categorical matching the quantized vectors of the codebook.
    This layer can be used to receive a high-level latent state from arbitrary input.
    It expects to be used in combination with a codebook instance that is managing
    the embeddings used for quantization."""

    def __init__(self, codebook: VQCodebook, name: str="vq_categorical"):
        super(VQCategorical, self).__init__(name=name)
        self.codebook = codebook

    def build(self, input_shape: Iterable[int]):
        self.codebook.init_codebook(input_shape)

    def call(self, inputs: tf.Tensor):
        categoricals_sparse = self.codebook.most_similar_embeddings(inputs)
        return tf.one_hot(categoricals_sparse, self.codebook.num_classes)


class VQCombined(Layer):
    def __init__(
            self, num_classifications: int, num_classes: int,
            name: str="vq_combined"):
        super().__init__(name=name)
        self.vq_codebook = VQCodebook(num_classifications, num_classes, name=f"{name}_codebook")
        self.vq_categorical = VQCategorical(self.vq_codebook, name=f"{name}_categorical")

    def build(self, input_shape):
        self.vq_categorical.build(input_shape)

    def call(self, inputs):
        categoricals = self.vq_categorical(inputs)
        quantized = self.vq_codebook(categoricals)
        return quantized, categoricals

In [5]:
def create_encoder(settings: DreamerSettings) -> Model:
    model_in = Input(settings.obs_dims, name="obs")
    #norm_img = Lambda(lambda x: x * 2.0 - 1.0)
    cnn_1 = Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_2 = Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_3 = Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_4 = Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    drop_1 = Dropout(rate=settings.dropout_rate)
    drop_2 = Dropout(rate=settings.dropout_rate)
    drop_3 = Dropout(rate=settings.dropout_rate)

    #img_in = norm_img(model_in)
    z_enc = cnn_4(drop_3(cnn_3(drop_2(cnn_2(drop_1(cnn_1(model_in)))))))
    return Model(inputs=model_in, outputs=z_enc, name="encoder_model")


def create_decoder(settings: DreamerSettings) -> Model:
    model_in = Input(settings.obs_enc_dims, name="obs_enc")
    cnn_1 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_2 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_3 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_4 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same", activation="elu")
    cnn_5 = Conv2D(settings.obs_dims[-1], (1, 1), padding="same", activation="linear")
    drop_1 = Dropout(rate=settings.dropout_rate)
    drop_2 = Dropout(rate=settings.dropout_rate)
    drop_3 = Dropout(rate=settings.dropout_rate)
    scale_255 = Lambda(lambda x: x * 255)

    model_out = scale_255(cnn_5(cnn_4(drop_3(cnn_3(drop_2(cnn_2(drop_1(cnn_1(model_in)))))))))
    return Model(inputs=model_in, outputs=model_out, name="decoder_model")


def create_vqvae(settings: DreamerSettings):
    encoder = create_encoder(settings)
    decoder = create_decoder(settings)
    vq = VQCombined(settings.repr_dims[0], settings.repr_dims[1], name="vq_z")

    obs_in = Input(settings.obs_dims, name="obs")
    z_enc = encoder(obs_in)
    z_quant, z_cat = vq(z_enc)
    obs_reconst = decoder(z_enc + tf.stop_gradient(z_quant - z_enc))
    committment_loss = tf.reduce_mean((tf.stop_gradient(z_quant) - z_enc) ** 2)
    codebook_loss = tf.reduce_mean((z_quant - tf.stop_gradient(z_enc)) ** 2)
    vqvae_loss = settings.committment_cost * committment_loss + codebook_loss

    z_cat_in = Input(settings.repr_dims)
    z_hat_quant = vq.vq_codebook(z_cat_in)
    obs_dream = decoder(z_hat_quant)

    vqvae = Model(inputs=obs_in, outputs=obs_reconst)
    vqvae.add_loss(vqvae_loss)
    obs_encoder = Model(inputs=obs_in, outputs=[z_quant, z_cat])
    dream_render = Model(inputs=z_cat_in, outputs=obs_dream)
    return vqvae, obs_encoder, dream_render, vq.vq_codebook


def create_vqrnn(settings: DreamerSettings):
    rnn = GRU(settings.hidden_dims[0], return_state=True)
    vq = VQCombined(settings.repr_dims[0], settings.repr_dims[1], name="vq_h")

    h0_quant_in = Input(settings.hidden_dims)
    x_in = Input([1] + settings.hidden_dims)

    _, h1_enc = rnn(x_in, initial_state=h0_quant_in)
    h1_quant, h1_cat = vq(h1_enc)
    h1_st_out = h1_enc + tf.stop_gradient(h1_quant - h1_enc)

    vqrnn = Model(inputs=[x_in, h0_quant_in], outputs=[h1_quant, h1_cat])
    vqrnn_train = Model(inputs=[x_in, h0_quant_in], outputs=[h1_enc, h1_quant, h1_cat, h1_st_out])
    return vqrnn, vqrnn_train, vq.vq_codebook


def create_vqrnn_embedding(
        settings: DreamerSettings, z_codebook: VQCodebook, h_codebook: VQCodebook):
    concat = Concatenate()
    flatten = Flatten()
    expand_timeseries = Reshape([-1] + settings.hidden_dims)
    fuse_z_a = Dense(settings.hidden_dims[-1], activation="linear")

    action_in = Input(settings.action_dims)
    h_cat_in = Input(settings.repr_dims)
    z_cat_in = Input(settings.repr_dims)

    z_quant = z_codebook(z_cat_in)
    h_quant = h_codebook(h_cat_in)
    x = expand_timeseries(fuse_z_a(concat((flatten(z_quant), action_in))))
    embed_ahz_cat = Model(inputs=[action_in, h_cat_in, z_cat_in], outputs=[x, h_quant])

    z_quant_in = Input(settings.obs_enc_dims)
    x = expand_timeseries(fuse_z_a(concat((flatten(z_quant_in), action_in))))
    fuse_az_rnn_input = Model(inputs=[action_in, z_quant_in], outputs=[x])
    return embed_ahz_cat, fuse_az_rnn_input


def create_dream_trans(settings: DreamerSettings, z_codebook: VQCodebook) -> Model:
    model_in = Input(settings.hidden_dims, name="h1_quant")
    dense1 = Dense(400, activation="relu")
    dense2 = Dense(400, activation="relu")
    dense3 = Dense(settings.repr_dims_flat)
    reshape = Reshape(settings.repr_dims)
    st_cat = STOneHotCategorical(settings.repr_dims[1])

    z1_cat = st_cat(reshape(dense3(dense2(dense1(model_in)))))
    z1_quant = z_codebook(z1_cat)
    return Model(inputs=model_in, outputs=[z1_cat, z1_quant], name="transition_model")


def create_reward_pred(settings: DreamerSettings) -> Model:
    flatten = Flatten()
    concat = Concatenate()
    dense_1 = Dense(64, "elu")
    dense_2 = Dense(64, "elu")
    reward = Dense(1, activation="linear")
    terminal = Dense(1, activation="sigmoid")
    stop_grads = Lambda(lambda x: tf.stop_gradient(x))

    h_quant_in = Input(settings.hidden_dims, name="h_quant_in")
    z_quant_in = Input(settings.obs_enc_dims, name="z_quant_in")

    prep = stop_grads(concat((flatten(z_quant_in), h_quant_in)))
    out = dense_2(dense_1(prep))
    reward_out = reward(out)
    terminal_out = terminal(out)
    return Model(inputs=[h_quant_in, z_quant_in], outputs=[reward_out, terminal_out], name="reward_model")

In [6]:
class DreamerModelComponents:
    def __init__(self, settings: DreamerSettings):
        self.settings = settings
        self.vqvae, self.obs_encoder, self.dream_render, self.z_codebook = create_vqvae(settings)
        self.vqrnn, self.vqrnn_train, self.h_codebook = create_vqrnn(settings)
        self.embed_ahz_cat, self.fuse_az_rnn_input = create_vqrnn_embedding(
            settings, self.z_codebook, self.h_codebook)
        self.dream_trans = create_dream_trans(settings, self.z_codebook)
        self.reward_pred = create_reward_pred(settings)

    def save_weights(self, directory: str):
        if not os.path.exists(directory):
            os.makedirs(directory)
        self.vqvae.save_weights(os.path.join(directory, f"vqvae.h5"))
        self.obs_encoder.save_weights(os.path.join(directory, f"obs_encoder.h5"))
        self.dream_render.save_weights(os.path.join(directory, f"dream_render.h5"))
        self.vqrnn.save_weights(os.path.join(directory, f"vqrnn.h5"))
        self.embed_ahz_cat.save_weights(os.path.join(directory, f"embed_ahz_cat.h5"))
        self.fuse_az_rnn_input.save_weights(os.path.join(directory, f"fuse_az_rnn_input.h5"))
        self.dream_trans.save_weights(os.path.join(directory, f"dream_trans.h5"))
        self.reward_pred.save_weights(os.path.join(directory, f"reward_pred.h5"))

    def load_weights(self, directory: str):
        if not os.path.exists(directory):
            raise FileNotFoundError(f"The specified directory '{directory}' does not exist!")
        self.vqvae.load_weights(os.path.join(directory, f"vqvae.h5"))
        self.obs_encoder.load_weights(os.path.join(directory, f"obs_encoder.h5"))
        self.dream_render.load_weights(os.path.join(directory, f"dream_render.h5"))
        self.vqrnn.load_weights(os.path.join(directory, f"vqrnn.h5"))
        self.embed_ahz_cat.load_weights(os.path.join(directory, f"embed_ahz_cat.h5"))
        self.fuse_az_rnn_input.load_weights(os.path.join(directory, f"fuse_az_rnn_input.h5"))
        self.dream_trans.load_weights(os.path.join(directory, f"dream_trans.h5"))
        self.reward_pred.load_weights(os.path.join(directory, f"reward_pred.h5"))

    def compose_models(self):
        s1 = Input(self.settings.obs_dims, name="s1")
        a0 = Input(self.settings.action_dims, name="a0")
        h0_cat = Input(self.settings.repr_dims, name="h0_cat")
        z0_cat = Input(self.settings.repr_dims, name="z0_cat")
        h0_quant = Input(self.settings.hidden_dims, name="h0_quant")
        z0_quant = Input(self.settings.obs_enc_dims, name="z0_quant")

        s1_hat = self.vqvae(s1)
        z1_quant, z1_cat = self.obs_encoder(s1)
        x0 = self.fuse_az_rnn_input((a0, tf.stop_gradient(z0_quant)))
        h1_enc, h1_quant, _, h1_st = self.vqrnn_train((x0, h0_quant))
        z1_cat_hat, _ = self.dream_trans(h1_st)
        r1_hat = self.reward_pred((h1_st, z1_quant))
        train_model = Model(
            inputs=[s1, a0, h0_quant, z0_quant],
            outputs=[h1_enc, h1_quant, s1_hat, r1_hat, z1_cat, z1_cat_hat],
            name="train_model")

        x0, h0_quant = self.embed_ahz_cat((a0, h0_cat, z0_cat))
        h1_quant, h1_cat = self.vqrnn((x0, h0_quant))
        z1_cat, z1_quant = self.dream_trans(h1_quant)
        r1_hat = self.reward_pred((h1_quant, z1_quant))
        dream_model = Model(
            inputs=[a0, h0_cat, z0_cat],
            outputs=[r1_hat, h1_cat, z1_cat],
            name="dream_model")

        z1_quant, z1_cat = self.obs_encoder(s1)
        x0, h0_quant = self.embed_ahz_cat((a0, h0_cat, z0_cat))
        h1_quant, h1_cat = self.vqrnn((x0, h0_quant))
        step_model = Model(
            inputs=[s1, a0, h0_cat, z0_cat],
            outputs=[h1_cat, z1_cat],
            name="step_model")

        z1_quant, _ = self.obs_encoder(s1)
        x0 = self.fuse_az_rnn_input((a0, tf.stop_gradient(z0_quant)))
        h1_quant, _ = self.vqrnn((x0, h0_quant))
        train_step_model = Model(
            inputs=[s1, a0, h0_quant, z0_quant],
            outputs=[h1_quant, z1_quant],
            name="step_model_diff")

        return train_model, dream_model, step_model, train_step_model, self.dream_render


class DreamerModel:
    def __init__(
            self, settings: DreamerSettings,
            model_comps: DreamerModelComponents=None,
            loss_logger: Callable[[float, float, float, float], None]=lambda l1, l2, l3, l4: None):
        self.settings = settings
        self.model_comps = model_comps if model_comps else DreamerModelComponents(settings)
        self.loss_logger = loss_logger
        self.optimizer = Adam(learning_rate=1e-4, epsilon=1e-5)
        self.train_model, self.dream_model, self.step_model, self.train_step_model, self.render_model = \
            self.model_comps.compose_models()

    def seed(self, seed: int):
        tf.random.set_seed(seed)

    def save(self, directory: str):
        self.model_comps.save_weights(directory)

    def load(self, directory: str):
        self.model_comps.load_weights(directory)

    def summary(self):
        self.train_model.summary()
        self.dream_model.summary()
        self.step_model.summary()
        self.render_model.summary()

    def bootstrap(self, initial_state):
        batch_size = initial_state.shape[0]
        a_in = tf.zeros([batch_size] + self.settings.action_dims)
        h_in = tf.zeros([batch_size] + self.settings.repr_dims)
        z_in = tf.zeros([batch_size] + self.settings.repr_dims)
        return self.step_model((initial_state, a_in, h_in, z_in))

    def step(self, s1, a0, h0, z0):
        return self.step_model((s1, a0, h0, z0))

    def dream_bootstrap(self, s0_init, a0_init):
        assert s0_init.shape[1] == a0_init.shape[1] + 1
        steps = a0_init.shape[1]
        h_in, z_in = self.bootstrap(s0_init[:, 0])
        for t in range(0, steps):
            s0, a0 = s0_init[:, t+1], a0_init[:, t]
            h_in, z_in = self.step(s0, a0, h_in, z_in)
        return h_in, z_in

    def dream_step(self, a, h, z):
        return self.dream_model((a, h, z))

    def render(self, z):
        return self.render_model(z)

    @tf.function
    def train(self, s0_init, a0_init, s1, a0, r1, t1):
        ALPHA = 0.2

        assert s0_init.shape[1] == a0_init.shape[1] + 1
        bootstrap_steps = a0_init.shape[1]
        batch_size = a0_init.shape[0]
        a_zero = tf.zeros([batch_size] + self.settings.action_dims)
        h0_quant = tf.zeros([batch_size] + self.settings.hidden_dims)
        z0_quant = tf.zeros([batch_size] + self.settings.obs_enc_dims)

        with tf.GradientTape() as tape:
            h0_quant, z0_quant = self.train_step_model((s0_init[:, 0], a_zero, h0_quant, z0_quant))
            for t in range(0, bootstrap_steps):
                h0_quant, z0_quant = self.train_step_model((s0_init[:, t+1], a0_init[:, t], h0_quant, z0_quant))
            h1_enc, h1_quant, s1_hat, (r1_hat, term_hat), z1_cat, z1_cat_hat \
                = self.train_model((s1, a0, h0_quant, z0_quant))

            committment_loss = tf.reduce_mean((tf.stop_gradient(h1_quant) - h1_enc) ** 2)
            codebook_loss = tf.reduce_mean((h1_quant - tf.stop_gradient(h1_enc)) ** 2)
            vqrnn_loss = self.settings.committment_cost * committment_loss + codebook_loss

            reconst_loss = tf.reduce_mean(MSE(tf.cast(s1, dtype=tf.float32) / 255.0, s1_hat / 255.0))
            repr_loss = tf.reduce_mean(KLDiv(tf.stop_gradient(z1_cat), z1_cat_hat))
                #+ (1 - ALPHA) * tf.reduce_mean(KLDiv(z1_cat, tf.stop_gradient(z1_cat_hat)))
            reward_loss = tf.reduce_mean(MSE(r1, r1_hat))
            term_loss = tf.reduce_mean(MSE(t1, term_hat))
            loss = vqvae_loss + vqrnn_loss + reconst_loss + reward_loss + term_loss + repr_loss

            grads = tape.gradient(loss, self.train_model.trainable_variables)
            self.optimizer.apply_gradients(zip(grads, self.train_model.trainable_variables))
            self.loss_logger(reconst_loss, repr_loss, reward_loss, term_loss)


@dataclass
class DreamerEnvWrapper(gym.Env):
    orig_env: gym.Env
    settings: DreamerSettings
    model: DreamerModel = field(default=None)
    render_output: RenderSubscriber = field(default=lambda frame_orig, frame_hall: None)
    collect_step: TrajectorySubscriber = field(default=lambda s1, a0, r1: None)
    # TODO: inject encode_obs as callable to support obs other than images

    action_space: gym.Space = field(init=False)
    observation_space: gym.Space = field(init=False)
    h0: np.ndarray = field(init=False)
    z0: np.ndarray = field(init=False)
    frame_orig: np.ndarray = field(init=False)

    def __post_init__(self):
        if not self.model:
            self.model = DreamerModel(self.settings)

        low = np.zeros(self.settings.repr_dims, dtype=np.float32)
        high = np.ones(self.settings.repr_dims, dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=low, high=high, dtype=np.float32)
        self.action_space = self.orig_env.action_space

    def reset(self):
        state = self.orig_env.reset()
        state = self._resize_image(state)
        self.h0, self.z0 = self.model.bootstrap(self._batch(state))
        self.frame_orig, repr = state, self._unbatch(self.z0)
        self.collect_step(state, None, None)
        return repr

    def step(self, action):
        state, reward, done, meta = self.orig_env.step(action)
        state = self._resize_image(state)
        self.h0, self.z0 = self.model.step(
            self._batch(state), self._batch(action), self.h0, self.z0)
        self.frame_orig, repr = state, self._unbatch(self.z0)
        self.collect_step(state, action, reward)
        return repr, reward, done, meta

    def render(self, mode="human"):
        # TODO: use decoder(trans(h1)) to see actual dream; decoder(z1) is VAE's latent state
        frame_hall = self.model.render(self.z0)
        self.render_output(self.frame_orig, self._unbatch(frame_hall))

    def seed(self, seed: int):
        self.orig_env.seed(seed)
        self.model.seed(seed)

    def _batch(self, arr: np.ndarray) -> np.ndarray:
        return np.expand_dims(arr, axis=0)

    def _unbatch(self, arr: np.ndarray) -> np.ndarray:
        return np.squeeze(arr, axis=0)

    def _resize_image(self, orig_image: np.ndarray) -> np.ndarray:
        width, height, _ = self.settings.obs_dims
        img = Image.fromarray(orig_image)
        img = img.resize((width, height), Image.Resampling.LANCZOS)
        return np.array(img)

In [7]:
def play_episode(
        env: gym.Env,
        actor: Callable[[Any], Any]=None,
        render: bool=False,
        max_steps: int=float("inf")):
    actor = actor if actor else lambda x: env.action_space.sample()
    render_frame = env.render if render else lambda: None
    state = env.reset()
    render_frame()
    done, i = False, 0
    while True:
        action = actor(state)
        state, __, done, ___ = env.step(action)
        render_frame()
        if done or i >= max_steps:
            break
        i += 1


def sample_trajectory(
        env: DreamerEnvWrapper,
        actor: Callable[[Any], Any],
        init_dt: int=5) -> Tuple[int, tf.data.Dataset]:
    traj_states = []
    traj_actions = []
    traj_rewards = []
    step = 0

    def reset_trajectory():
        nonlocal traj_states, traj_actions, traj_rewards, step
        traj_states = []
        traj_actions = []
        traj_rewards = []
        step = 0

    # info: append_timestep() yields a trajectory with
    #       t steps and t rewards; for bootstrapping,
    #       it records initial +5 states and +4 actions
    def append_timestep(s1, a, r):
        nonlocal step
        if step < init_dt:
            traj_states.append(s1)
            if step >= 1:
                traj_actions.append(a)
        else:
            traj_states.append(s1)
            traj_actions.append(a)
            traj_rewards.append(r)
        step += 1

    # info: handle the case when the trajectory
    #      terminated during the initial steps
    env.collect_step = append_timestep
    valid_traj = False
    while not valid_traj:
        play_episode(env, actor)
        valid_traj = step > init_dt
        if not valid_traj:
            reset_trajectory()
    env.collect_step = lambda s1, a, r: None

    ep_dt = len(traj_rewards)
    terms = np.zeros(ep_dt)
    terms[-1] = 1.0
    s0_init = np.array([traj_states[t:t+init_dt] for t in range(ep_dt)])
    a0_init = np.array([traj_actions[t:t+init_dt-1] for t in range(ep_dt)])

    s1 = np.array([traj_states[t+init_dt] for t in range(ep_dt)])
    a0 = np.array([traj_actions[t+init_dt-1] for t in range(ep_dt)])
    r1 = np.array(traj_rewards)

    return ep_dt, tf.data.Dataset.from_tensor_slices(tensors=(
        s0_init, a0_init, s1, a0, r1, terms))


def concat_datasets(datasets: List[tf.data.Dataset]) -> tf.data.Dataset:
    ds = datasets[0]
    for i in range(len(datasets) - 1):
        ds = ds.concatenate(datasets[i+1])
    return ds

In [8]:
orig_env = gym.make("ALE/MsPacman-v5")
settings = DreamerSettings([1], [64, 64, 3], [4, 4, 64], [32, 32], [512])
model_comps = DreamerModelComponents(settings)
model_comps.load_weights("dreamer_pretrained")
model = DreamerModel(settings)
env = DreamerEnvWrapper(orig_env, settings, model)

In [9]:
batch_size = 64
random_actor = lambda obs: env.action_space.sample()
trajs = [sample_trajectory(env, random_actor)[1] for i in tqdm(range(10))]
orig_dataset = concat_datasets(trajs)

100%|██████████| 10/10 [07:21<00:00, 44.17s/it]


In [10]:
def eval_vqvae(vqvae, dataset):

    def show_subplot(original, reconstructed):
        plt.subplot(1, 2, 1)
        plt.imshow(original)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(reconstructed)
        plt.title("Reconstructed")
        plt.axis("off")
        plt.show()

    ds_shuffled = dataset.unbatch().shuffle(100).batch(10)
    batch = next(iter(ds_shuffled))
    img_orig, _ = batch
    print("input shape", img_orig.shape)

    img_reconst = vqvae(img_orig)
    print("output shape", img_reconst.shape)
    print("reconst loss is", tf.reduce_mean((tf.cast(img_orig, dtype=tf.float32) - img_reconst)**2).numpy())
    print("codebook loss is", tf.reduce_sum(vqvae.losses).numpy())

    for orig, reconst in zip(img_orig.numpy(), img_reconst.numpy()):
        show_subplot(orig, reconst)

In [11]:
vqvae, obs_encoder, dream_render, z_codebook = create_vqvae(settings)
vqvae_dataset = orig_dataset.map(lambda s0_init, a0_init, s1, a0, r1, t1: (s1))
vqvae_dataset = vqvae_dataset.map(lambda s1: tf.cast(s1, tf.float32) / 255.0)        
vqvae_dataset = vqvae_dataset.map(lambda s1_norm: (s1_norm, s1_norm))
shuffled_train_dataset = vqvae_dataset.shuffle(100).batch(64)
vqvae.summary()
vqvae.compile(optimizer="adam", loss="mse", metrics=["mse"])
vqvae.fit(epochs=10000, x=shuffled_train_dataset)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 obs (InputLayer)               [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 encoder_model (Functional)     (None, 4, 4, 64)     112576      ['obs[0][0]']                    
                                                                                                  
 vq_z (VQCombined)              ((None, 4, 4, 64),   32768       ['encoder_model[0][0]']          
                                 (None, 32, 32))                                        

77/77 [==============================] - 2s 23ms/step - loss: 0.0081 - mse: 0.0073
Epoch 26/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0081 - mse: 0.0073
Epoch 27/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0068 - mse: 0.0060
Epoch 28/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0082 - mse: 0.0075
Epoch 29/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0065 - mse: 0.0058
Epoch 30/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0076 - mse: 0.0069
Epoch 31/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0074 - mse: 0.0067
Epoch 32/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0070 - mse: 0.0063
Epoch 33/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0064 - mse: 0.0058
Epoch 34/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0073 - mse: 0.0067
Epoch 35/10000
77/77 [=============

77/77 [==============================] - 2s 23ms/step - loss: 0.0069 - mse: 0.0058
Epoch 109/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0063 - mse: 0.0054
Epoch 110/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0059 - mse: 0.0051
Epoch 111/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0070 - mse: 0.0059
Epoch 112/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0061 - mse: 0.0051
Epoch 113/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0059 - mse: 0.0050
Epoch 114/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0056 - mse: 0.0047
Epoch 115/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0054 - mse: 0.0045
Epoch 116/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0052 - mse: 0.0044
Epoch 117/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0049 - mse: 0.0041
Epoch 118/10000
77/77 [===

77/77 [==============================] - 2s 22ms/step - loss: 0.0038 - mse: 0.0031
Epoch 191/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0038 - mse: 0.0032
Epoch 192/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0038 - mse: 0.0031
Epoch 193/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0038 - mse: 0.0031
Epoch 194/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0037 - mse: 0.0030
Epoch 195/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0039 - mse: 0.0032
Epoch 196/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0037 - mse: 0.0031
Epoch 197/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0037 - mse: 0.0031
Epoch 198/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0036 - mse: 0.0030
Epoch 199/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0037 - mse: 0.0030
Epoch 200/10000
77/77 [===

77/77 [==============================] - 2s 23ms/step - loss: 0.0068 - mse: 0.0048
Epoch 273/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0065 - mse: 0.0046
Epoch 274/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0067 - mse: 0.0048
Epoch 275/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0064 - mse: 0.0045
Epoch 276/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0065 - mse: 0.0046
Epoch 277/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0063 - mse: 0.0045
Epoch 278/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0062 - mse: 0.0045
Epoch 279/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0061 - mse: 0.0044
Epoch 280/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0062 - mse: 0.0045
Epoch 281/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0060 - mse: 0.0043
Epoch 282/10000
77/77 [===

...

77/77 [==============================] - 2s 23ms/step - loss: 0.0396 - mse: 0.0359
Epoch 437/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0391 - mse: 0.0358
Epoch 438/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0382 - mse: 0.0350
Epoch 439/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0385 - mse: 0.0353
Epoch 440/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0372 - mse: 0.0343
Epoch 441/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0368 - mse: 0.0343
Epoch 442/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0363 - mse: 0.0339
Epoch 443/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0387 - mse: 0.0348
Epoch 444/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0364 - mse: 0.0332
Epoch 445/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0354 - mse: 0.0331
Epoch 446/10000
77/77 [===

77/77 [==============================] - 2s 23ms/step - loss: 0.0448 - mse: 0.0376
Epoch 519/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0435 - mse: 0.0371
Epoch 520/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0425 - mse: 0.0366
Epoch 521/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0414 - mse: 0.0361
Epoch 522/10000
77/77 [==============================] - 2s 26ms/step - loss: 0.0405 - mse: 0.0357
Epoch 523/10000
77/77 [==============================] - 2s 26ms/step - loss: 0.0397 - mse: 0.0353
Epoch 524/10000
77/77 [==============================] - 2s 27ms/step - loss: 0.0389 - mse: 0.0349
Epoch 525/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0381 - mse: 0.0346
Epoch 526/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0375 - mse: 0.0342
Epoch 527/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0369 - mse: 0.0339
Epoch 528/10000
77/77 [===

77/77 [==============================] - 2s 24ms/step - loss: 0.0266 - mse: 0.0264
Epoch 601/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0266 - mse: 0.0264
Epoch 602/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0264 - mse: 0.0262
Epoch 603/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0265 - mse: 0.0263
Epoch 604/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0264 - mse: 0.0262
Epoch 605/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0264 - mse: 0.0262
Epoch 606/10000
77/77 [==============================] - 2s 25ms/step - loss: 0.0266 - mse: 0.0264
Epoch 607/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0262 - mse: 0.0260
Epoch 608/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0262 - mse: 0.0260
Epoch 609/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0263 - mse: 0.0261
Epoch 610/10000
77/77 [===

77/77 [==============================] - 2s 23ms/step - loss: 0.0408 - mse: 0.0281
Epoch 683/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0388 - mse: 0.0274
Epoch 684/10000
77/77 [==============================] - 2s 25ms/step - loss: 0.0371 - mse: 0.0268
Epoch 685/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0355 - mse: 0.0262
Epoch 686/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0341 - mse: 0.0257
Epoch 687/10000
77/77 [==============================] - 2s 22ms/step - loss: 0.0328 - mse: 0.0252
Epoch 688/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0316 - mse: 0.0246
Epoch 689/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0304 - mse: 0.0241
Epoch 690/10000
77/77 [==============================] - 2s 23ms/step - loss: 0.0294 - mse: 0.0236
Epoch 691/10000
77/77 [==============================] - 2s 24ms/step - loss: 0.0285 - mse: 0.0233
Epoch 692/10000
77/77 [===

KeyboardInterrupt: 

In [ ]:
eval_vqvae(vqvae, shuffled_train_dataset)